In [71]:
import xml.etree.ElementTree as ET
import os
import pprint

# establish path of XML file
path = "../NASA/NumericalData/ACE/Attitude/Definitive/"
fullname = os.path.join(path, "PT1H.xml")
tree = ET.parse(fullname)
root = tree.getroot()
# root[1] = NumericalData
# root = Spase
# ET.dump(tree)
# for event, element in ET.iterparse(fullname):
    #print(event, element.tag)
ET.register_namespace("spase","http://www.spase-group.org/data/schema")
ns = {"spase":"http://www.spase-group.org/data/schema"}

In [23]:
# obtain ResourceID
target_tag = "{http://www.spase-group.org/data/schema}ResourceID"
for descendant in root[1].iter(target_tag):
    print(descendant.text)

spase://NASA/NumericalData/ACE/Attitude/Definitive/PT1H


In [78]:
# iterate thru children to locate ResourceHeader
for child in root[1]:
    print(child.tag)
    if child.tag.endswith("ResourceHeader"):
        targetChild = child
    
# obtain Author, Publication Date, Publisher, and Dataset Name

# define vars
author="None" 
pubDate="None"
pub = "None"
dataset = "None"
PI = "None"

# iterate thru ResourceHeader
for child in targetChild:
    #print(child.tag)
    # find backup Dataset Name
    if child.tag.endswith("ResourceName"):
        targetChild = child
        dataset = child.text
    # find Persistent Identifier
    if child.tag.endswith("DOI"):
        PI = child.text
    # find Publication Info
    elif child.tag.endswith("PublicationInfo"):
        targetChild = child
        #break

# access Publication Info
for child in targetChild:
    if child.tag.endswith("Authors"):
        author = child.text
    elif child.tag.endswith("PublicationDate"):
        pubDate = child.text
    elif child.tag.endswith("PublishedBy"):
        pub = child.text
    elif child.tag.endswith("Title"):
        dataset = child.text

pubYear = pubDate[0:4]
print(author)
print(pubYear)
print(pub)
print(dataset)
print(PI)

{http://www.spase-group.org/data/schema}ResourceID
{http://www.spase-group.org/data/schema}ResourceHeader
{http://www.spase-group.org/data/schema}AccessInformation
{http://www.spase-group.org/data/schema}AccessInformation
{http://www.spase-group.org/data/schema}ProcessingLevel
{http://www.spase-group.org/data/schema}InstrumentID
{http://www.spase-group.org/data/schema}MeasurementType
{http://www.spase-group.org/data/schema}TemporalDescription
{http://www.spase-group.org/data/schema}ObservedRegion
{http://www.spase-group.org/data/schema}ObservedRegion
{http://www.spase-group.org/data/schema}ObservedRegion
{http://www.spase-group.org/data/schema}Parameter
{http://www.spase-group.org/data/schema}Parameter
{http://www.spase-group.org/data/schema}Parameter
{http://www.spase-group.org/data/schema}Parameter
ACE flight operations team (FOT), and R. Candey
2022
NASA Space Physics Data Facility
ACE Definitive Attitude, Hourly Values, Direction Cosines in RTN, GSE, and J2000 GCI Coordinates
https

In [83]:
# dictionary labled by the Access Rights which will store all URLs and their Product Keys if given
Open = {}

# iterate thru children to locate Access Information
for child in root[1]:
    if child.tag.endswith("AccessInformation"):
        targetChild = child
        # iterate thru children to locate AccessURL and RepositoryID
        for child in targetChild:
            if child.tag.endswith("AccessURL"):
                targetChild = child
                # iterate thru children to locate URL
                for child in targetChild:
                    if child.tag.endswith("URL"):
                        url = child.text
                        Open[url] = "No_product_key"
                    # check if URL has a product key
                    elif child.tag.endswith("ProductKey"):
                        prodKey = child.text
                        Open[url] = prodKey
            # find backup Publisher if needed
            elif pub == "None":
                if child.tag.endswith("RepositoryID"):
                    # use partition to split text by Repository/ and assign only the text after it to pub 
                    before, sep, after = child.text.partition("Repository/")
                    pub = after
            # continue to check for additional AccessURLs            
            continue
    # continue to check for additional Access Informations
    continue

print("The data links and their associated product keys with an Open License are:")
pprint.pprint(Open)

The data links and their associated product keys with an Open License are:
{'ftps://spdf.gsfc.nasa.gov/pub/data/ace/orbit/level_2_cdaweb/def_at/': 'No_product_key',
 'https://cdaweb.gsfc.nasa.gov/cgi-bin/eval2.cgi?dataset=AC_AT_DEF&index=sp_phys': 'AC_AT_DEF',
 'https://cdaweb.gsfc.nasa.gov/hapi': 'AC_AT_DEF',
 'https://spdf.gsfc.nasa.gov/pub/data/ace/orbit/level_2_cdaweb/def_at/': 'No_product_key'}
